In [1]:
import pandas as pd 
# Load the data
dsProgReports = pd.read_csv("./NYC_Schools/School_Progress_Reports_-_All_Schools_-_2009-10.csv")
dsDistrict = pd.read_csv("./NYC_Schools/School_District_Breakdowns.csv")
dsClassSize = pd.read_csv("./NYC_Schools/2009-10_Class_Size_-_School-level_Detail.csv")
dsAttendEnroll = pd.read_csv("./NYC_Schools/School_Attendance_and_Enrollment_Statistics_by_District__2010-11_.csv")[:-2]#last two rows are bad
dsSATs = pd.read_csv("./NYC_Schools/SAT__College_Board__2010_School_Level_Results.csv") # Dependent

In [2]:
dsSATs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460 entries, 0 to 459
Data columns (total 6 columns):
DBN                      460 non-null object
School Name              460 non-null object
Number of Test Takers    460 non-null object
Critical Reading Mean    460 non-null object
Mathematics Mean         460 non-null object
Writing Mean             460 non-null object
dtypes: object(6)
memory usage: 25.2+ KB


In [3]:
pd.DataFrame(data=[dsProgReports['DBN'].take(range(5)), dsSATs['DBN'].take(range(5)), dsClassSize['SCHOOL CODE'].take(range(5))])

,0,1,2,3,4
DBN,01M015,01M019,01M020,01M034,01M063
DBN,01M292,01M448,01M450,01M458,01M509
SCHOOL CODE,M015,M015,M015,M015,M015


In [4]:
# Strip the first two characters off the DBNs so we can join to School Code
dsProgReports.DBN = dsProgReports.DBN.map(lambda x: x[2:])
dsSATs.DBN = dsSATs.DBN.map(lambda x: x[2:])
# We can now see the keys match
pd.DataFrame(data=[dsProgReports['DBN'].take(range(5)), dsSATs['DBN'].take(range(5)), dsClassSize['SCHOOL CODE'].take(range(5))])

,0,1,2,3,4
DBN,M015,M019,M020,M034,M063
DBN,M292,M448,M450,M458,M509
SCHOOL CODE,M015,M015,M015,M015,M015


In [5]:
# Show the key mismatches
# For variety's sake, using slicing ([:3]) syntax instead of .take()
pd.DataFrame(data=[dsProgReports['DISTRICT'][:3], dsDistrict['JURISDICTION NAME'][:3], dsAttendEnroll['District'][:3]])

,0,1,2
DISTRICT,1,1,1
JURISDICTION NAME,CSD 01 Manhattan,CSD 02 Manhattan,CSD 03 Manhattan
District,DISTRICT 01,DISTRICT 02,DISTRICT 03


In [6]:
# Extract well-formed district key values
# Note the astype(int) at the end of these lines to coerce the column to a numeric type
import re
dsDistrict['JURISDICTION NAME'] = dsDistrict['JURISDICTION NAME'].map(lambda x: re.match(r'([A-Za-z]*\s)([0-9]*)', x).group(2)).astype(int)
dsAttendEnroll.District = dsAttendEnroll.District.map(lambda x: x[-2:]).astype(int)
# We can now see the keys match
pd.DataFrame(data=[dsProgReports['DISTRICT'][:3], dsDistrict['JURISDICTION NAME'][:3], dsAttendEnroll['District'][:3]])

,0,1,2
DISTRICT,1,1,1
JURISDICTION NAME,1,2,3
District,1,2,3


In [7]:
# Reindexing
dsProgReports = dsProgReports.set_index('DBN')
dsDistrict = dsDistrict.set_index('JURISDICTION NAME')
dsClassSize = dsClassSize.set_index('SCHOOL CODE')
dsAttendEnroll = dsAttendEnroll.set_index('District')
dsSATs = dsSATs.set_index('DBN')

In [8]:
# We can seet the bad value
dsSATs['Critical Reading Mean'].take(range(5))

DBN
M292    391
M448    394
M450    418
M458    385
M509      s
Name: Critical Reading Mean, dtype: object

In [9]:
# Now we filter it out
# We create a boolean vector mask. Open question as to whether this semantically ideal...
mask = dsSATs['Number of Test Takers'].map(lambda x: x != 's')
dsSATs = dsSATs[mask]
# Cast fields to integers. Ideally we should not need to be this explicit
dsSATs['Number of Test Takers'] = dsSATs['Number of Test Takers'].astype(int)
dsSATs['Critical Reading Mean'] = dsSATs['Critical Reading Mean'].astype(int)
dsSATs['Writing Mean'] = dsSATs['Writing Mean'].astype(int)
# We can see those values are gone
dsSATs['Critical Reading Mean'].take(range(5))

DBN
M292    391
M448    394
M450    418
M458    385
M515    314
Name: Critical Reading Mean, dtype: int64

In [10]:
# The shape of the data 
print dsClassSize.columns
print dsClassSize.take([0,1,10]).values

Index([u'BORO', u'CSD', u'SCHOOL NAME', u'GRADE ', u'PROGRAM TYPE',
       u'CORE SUBJECT (MS CORE and 9-12 ONLY)',
       u'CORE COURSE (MS CORE and 9-12 ONLY)', u'SERVICE CATEGORY(K-9* ONLY)',
       u'NUMBER OF CLASSES', u'TOTAL REGISTER', u'AVERAGE CLASS SIZE',
       u'SIZE OF SMALLEST CLASS', u'SIZE OF LARGEST CLASS', u'DATA SOURCE',
       u'SCHOOLWIDE PUPIL-TEACHER RATIO'],
      dtype='object')
[['M' 1 'P.S. 015 ROBERTO CLEMENTE' '0K' 'GEN ED' '-' '-' '-' 1.0 21.0 21.0
  21.0 21.0 'ATS' nan]
 ['M' 1 'P.S. 015 ROBERTO CLEMENTE' '0K' 'CTT' '-' '-' '-' 1.0 21.0 21.0
  21.0 21.0 'ATS' nan]
 ['M' 1 'P.S. 015 ROBERTO CLEMENTE' nan nan nan nan nan nan nan nan nan nan
  nan 8.9]]


In [11]:
# Extracting the Pupil-Teacher Ratio
# Take the column
dsPupilTeacher = dsClassSize.filter(['SCHOOLWIDE PUPIL-TEACHER RATIO'])
# And filter out blank rows
mask = dsPupilTeacher['SCHOOLWIDE PUPIL-TEACHER RATIO'].map(lambda x: x>0)
dsPupilTeacher = dsPupilTeacher[mask]
# Then drop from the original dataset
dsClassSize = dsClassSize.drop('SCHOOLWIDE PUPIL-TEACHER RATIO', axis=1)
# Drop non-numeric fields
dsClassSize = dsClassSize.drop(['BORO','CSD','SCHOOL NAME','GRADE ','PROGRAM TYPE',\
'CORE SUBJECT (MS CORE and 9-12 ONLY)','CORE COURSE (MS CORE and 9-12 ONLY)',\
'SERVICE CATEGORY(K-9* ONLY)','DATA SOURCE'], axis=1)
# Build feature from dsClassSize
# In this case, we'll take the max, min and mean
# Semantically equivalent to seleect min(*), max(*), mean(*) from dsClassSize group by SCHOOL NAME
# Note that SCHOOL NAME is not referenced explicityly below because it is the index of the dataframe
import numpy as np
grouped = dsClassSize.groupby(level=0)
dsClassSize = grouped.aggregate(np.max).\
    join(grouped.aggregate(np.min), lsuffix=".max").\
    join(grouped.aggregate(np.mean), lsuffix=".min", rsuffix=".mean").\
    join(dsPupilTeacher)
print dsClassSize.columns
#print grouped.describe()

Index([u'NUMBER OF CLASSES.max', u'TOTAL REGISTER.max',
       u'AVERAGE CLASS SIZE.max', u'SIZE OF SMALLEST CLASS.max',
       u'SIZE OF LARGEST CLASS.max', u'NUMBER OF CLASSES.min',
       u'TOTAL REGISTER.min', u'AVERAGE CLASS SIZE.min',
       u'SIZE OF SMALLEST CLASS.min', u'SIZE OF LARGEST CLASS.min',
       u'NUMBER OF CLASSES.mean', u'TOTAL REGISTER.mean',
       u'AVERAGE CLASS SIZE.mean', u'SIZE OF SMALLEST CLASS.mean',
       u'SIZE OF LARGEST CLASS.mean', u'SCHOOLWIDE PUPIL-TEACHER RATIO'],
      dtype='object')


In [12]:
mask = dsProgReports['SCHOOL LEVEL*'].map(lambda x: x == "High School")
dsProgReports = dsProgReports[mask]

In [13]:
final = dsSATs.join(dsClassSize).\
join(dsProgReports).\
merge(dsDistrict, left_on='DISTRICT', right_index=True).\
merge(dsAttendEnroll, left_on='DISTRICT', right_index=True)

In [14]:
final.dtypes[final.dtypes.map(lambda x: x=='object')]

School Name                      object
Mathematics Mean                 object
SCHOOL                           object
PRINCIPAL                        object
PROGRESS REPORT TYPE             object
SCHOOL LEVEL*                    object
2009-2010 OVERALL GRADE          object
2009-2010 ENVIRONMENT GRADE      object
2009-2010 PERFORMANCE GRADE      object
2009-2010 PROGRESS GRADE         object
2008-09 PROGRESS REPORT GRADE    object
YTD % Attendance (Avg)           object
dtype: object

In [15]:
# Just drop string columns
# In theory we could build features out of some of these, but it is impractical here
final = final.drop(['School Name','SCHOOL','PRINCIPAL','SCHOOL LEVEL*','PROGRESS REPORT TYPE'],axis=1)
# Remove % signs and convert to float
final['YTD % Attendance (Avg)'] = final['YTD % Attendance (Avg)'].map(lambda x: x.replace("%", "")).astype(float)
# The last few columns we still have to deal with
final.dtypes[final.dtypes.map(lambda x: x=='object')]

Mathematics Mean                 object
2009-2010 OVERALL GRADE          object
2009-2010 ENVIRONMENT GRADE      object
2009-2010 PERFORMANCE GRADE      object
2009-2010 PROGRESS GRADE         object
2008-09 PROGRESS REPORT GRADE    object
dtype: object

In [16]:
gradeCols = ['2009-2010 OVERALL GRADE','2009-2010 ENVIRONMENT GRADE','2009-2010 PERFORMANCE GRADE','2009-2010 PROGRESS GRADE','2008-09 PROGRESS REPORT GRADE']
grades = np.unique(final[gradeCols].values) # [nan, A, B, C, D, F]
for c in gradeCols:
    for g in grades:
        final = final.join(pd.Series(data=final[c].map(lambda x: 1 if x is g else 0), name=c + "_is_" + str(g))) 
final = final.drop(gradeCols, axis=1)
# To generate csv files
final.drop(['Critical Reading Mean', 'Mathematics Mean', 'Writing Mean'], axis=1).to_csv('./NYC_Schools/train.csv')
final.filter(['Critical Reading Mean', 'Mathematics Mean', 'Writing Mean']).to_csv('./NYC_Schools/target.csv')

/Users/Jerry/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [17]:
from __future__ import print_function
import math
from sklearn.ensemble import RandomForestRegressor

final = final.dropna(axis=0)
target = final.filter(['Critical Reading Mean'])
#We drop all three dependent variables because we don't want them used when trying to make a prediction.
train = final.drop(['Critical Reading Mean','Writing Mean','Mathematics Mean'],axis=1)
model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
model.fit(train, target)

predictions = np.array(model.predict(train))
rmse = math.sqrt(np.mean((np.array(target.values) - predictions)**2))
imp = sorted(zip(train.columns, model.feature_importances_), key=lambda tup: tup[1], reverse=True)
print ("RMSE: " + str(rmse))
print ("10 Most Important Variables:")
print (*imp[:10], sep='\n', end='\n')

RMSE: 81.4630655353
10 Most Important Variables:
('PEER INDEX*', 0.84144300585913601)
('2009-2010 ENVIRONMENT CATEGORY SCORE', 0.022807061336079407)
('SCHOOLWIDE PUPIL-TEACHER RATIO', 0.0095309032653001606)
('2009-2010 PERFORMANCE CATEGORY SCORE', 0.0091793371545553955)
('Number of Test Takers', 0.0084008086535322119)
('TOTAL REGISTER.min', 0.0068761259834743561)
('DISTRICT', 0.0056017988794162341)
('NUMBER OF CLASSES.mean', 0.0052155821376175934)
('AVERAGE CLASS SIZE.mean', 0.0049883280491015364)
('2009-2010 PROGRESS CATEGORY SCORE', 0.0049757137173666761)


/Users/Jerry/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
